In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from mydataset import MyDataSet
from myloss import TripletLoss

torch.manual_seed(10)

from torchvision import models

In [2]:
EPOCH = 25
BATCH_SIZE = 4
LR = 0.001

In [3]:
model = models.vgg16(pretrained=True)
# print(model)
# print(model.parameters)    
model.classifier = nn.Sequential(*list(model.classifier.children())[:-5])

for parma in model.parameters():
        parma.requires_grad = False
    
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [11]:
def hook (module,input,output):
    print(output.size())
# 注册
handle = model.features[0].register_forward_hook(hook)
# 删除句柄
handle.remove()

# torch.nn.Module.register_backward_hook(hook)
# hook(module, grad_input, grad_output) -> Tensor or None
#model = alexnet(pretrained=True).to(device)
outputs = []
def hook (module,input,output):
    outputs.append(output)
    print(len(outputs))

handle = model.features[0].register_backward_hook(hook)


In [4]:
input_size = 224
transform = transforms.Compose([   
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_set = MyDataSet('./Birds/train/',transform=transform, labels=None)
train_loader = Data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_set = MyDataSet('./Birds/test/', transform=transform, labels=None)
test_loader = Data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [5]:
print(train_set.classes)
print(train_set.class_to_idx)
print(train_set.__len__)

print(test_set.classes)
print(test_set.class_to_idx)
print(test_set.__len__)

['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird']
{'001.Black_footed_Albatross': 0, '002.Laysan_Albatross': 1, '003.Sooty_Albatross': 2, '004.Groove_billed_Ani': 3, '005.Crested_Auklet': 4, '006.Least_Auklet': 5, '007.Parakeet_Auklet': 6, '008.Rhinoceros_Auklet': 7, '009.Brewer_Blackbird': 8, '010.Red_winged_Blackbird': 9}
<bound method MyDataSet.__len__ of <mydataset.MyDataSet object at 0x00000200FDDF3D68>>
['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird']
{'001.Black_footed_Albatross': 0, '002.Laysan_Albatross': 1, '003.Sooty_Albatross': 2, '004.Groove_billed_Ani': 3, '005.Crested_Auklet': 4, '006.Lea

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
cross_loss = nn.CrossEntropyLoss()   # the target label is not one-hotted
triplet_loss = TripletLoss(0.5) # 选择损失函数
alpha = 0.5

for epoch in range(EPOCH):
    print('EPOCH ' + str(epoch))
    for step, (anchor, positive, negative) in enumerate(train_loader):
        anchor_output = model(anchor[0])
        print(anchor_output)
        var = Variable(anchor_output)
        print(var.size())
        positive_output = model(positive[0])
        negative_output = model(negative[0])
        
        triplet_result = triplet_loss(anchor_output[1], positive_output[1], negative_output[1])
        cross_result = cross_loss(anchor_output[0], anchor[1])
        loss = alpha * triplet_result + (1-alpha) * cross_result
                                       
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            print('训练三元损失：', triplet_result)
            print('训练交叉熵损失：', cross_result)
            
            correct = 0
            total = 0
            
            for _, (test, _, _) in enumerate(test_loader):
                test_output = model(test[0])[0]
                predicted = torch.max(test_output, 1).indices
                labels = test[1]
                total += labels.size(0)
                correct += (predicted == labels).sum()
                
            print('Accuracy on the test images: %d %%' % (100 * correct / total))

EPOCH 0
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.8024, 1.0499,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1772, 2.2189, 0.3360,  ..., 0.0000, 0.2604, 0.0000]])
torch.Size([4, 4096])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)